In [1]:
from sibyl.core import Sibyl

In [2]:
config = {
    "mongodb": {
        "db": "housing",
        "host": "localhost",
        "port": 27017,
        "username": None,
        "password": None,
    },
    "log_filename": "test.csv",
    "feature_distribution_location": None,
    "flask": {},
}
explorer = Sibyl(config, docker=False)
app = explorer._init_flask_app("test")
client = app.test_client()

In [3]:
# TEST ENTITY APIs

response = client.get("/api/v1/entities/")
assert response.status_code == 200
assert 'entities' in response.json
print("Number of entities:",   len(response.json['entities']))

eid = response.json["entities"][0]["eid"]

response = client.get("/api/v1/entities/" + eid + "/")
assert response.status_code == 200
assert response.json["eid"] == eid
assert "features" in response.json
assert "property" in response.json
sample_entity = response.json

Number of entities: 1460


In [4]:
# TEST FEATURE APIs

response = client.get("/api/v1/features/")
assert response.status_code == 200
assert "features" in response.json
sample_feature = response.json['features'][0]

response = client.get("/api/v1/features/" + sample_feature["name"] + "/")
assert response.status_code == 200
assert "name" in response.json
print("Sample feature:", response.json)

response = client.get("/api/v1/categories/")
assert response.status_code == 200
assert "categories" in response.json
print("Sample category", response.json["categories"][0])

Sample feature: {'name': 'MSSubClass', 'description': ' Identifies the type of dwelling involved in the sale.', 'type': 'category', 'negated_description': None, 'category': 'general'}
Sample category {'name': 'general', 'color': None, 'abbreviation': None}


In [5]:
# TEST MODEL APIs
response = client.get("/api/v1/models/")
assert response.status_code == 200
sample_model = response.json["models"][0]
model_id = sample_model["id"]

response = client.get("/api/v1/models/" + model_id + "/")
assert response.status_code == 200
print("Sample model:", response.json)

response = client.get("/api/v1/importance/?model_id=" + model_id)
assert response.status_code == 200
assert "importances" in response.json
print("Sample importance", next(iter(response.json["importances"].items())))

response = client.get("/api/v1/prediction/?model_id=" + model_id + "&eid=" + eid)
assert response.status_code == 200
print("Sample prediction:", response.json)
prediction = response.json["output"]

Sample model: {'id': '644aae7eefd589518033bd96', 'name': 'placeholder', 'description': 'placeholder', 'performance': 'placeholder'}
Sample importance ('Id', 23.334827162604206)
Sample prediction: {'output': 213151.6906855132}


In [10]:
# TEST COMPUTING

response = client.post("/api/v1/contributions/", json={"eid": eid, "model_id": model_id})
assert response.status_code == 200
assert "contributions" in response.json
print("Sample contribution", next(iter(response.json["contributions"].items())))

#response = client.post("/api/v1/prediction_count/", json={"prediction": prediction, "model_id": model_id})
#response.json

features_options = list(sample_entity["features"].keys())
changes = [(features_options[0], sample_entity["features"][features_options[0]]),
           (features_options[1], sample_entity["features"][features_options[1]])]
response = client.post("/api/v1/modified_prediction/",json={"eid": eid, "model_id": model_id, "changes": changes})
assert response.status_code == 200
assert "prediction" in response.json
print("Sample modified prediction:", response.json)

response = client.post("/api/v1/single_change_predictions/", json={"eid": eid, "model_id": model_id, "changes": changes})
assert response.status_code == 200
response.json

could not convert string to float: 'RL'
Traceback (most recent call last):
  File "C:\Users\Ola\Documents\github\sibyl-api\sibyl\resources\computing.py", line 82, in post
    change[1] = float(change[1])
ValueError: could not convert string to float: 'RL'


Sample contribution ('MSSubClass', 100.62363952750484)
Sample modified prediction: {'prediction': 213151.6906855132}


AssertionError: 